In [1]:
import pandas as pd

# Load the interpolated data
df = pd.read_csv("data/processed/interpolated_cgm.csv", parse_dates=["time"])

ModuleNotFoundError: No module named 'pandas'

In [ ]:


# Define a threshold for acceptable NaNs (e.g., 10% of a 24-hour period = 288 * 0.1 = 28 max NaNs)
nan_threshold = 288*0.2  # Adjust based on your tolerance

# Group by ID and date
df["date"] = df["time"].dt.date
valid_days = df.groupby(["ID", "date"]).apply(lambda group: group["glc"].isna().sum() <= nan_threshold)

# Initial number of days
initial_days = df.groupby(["ID", "date"]).ngroups
print(f"Initial number of days: {initial_days}")

# Filter out invalid days
valid_days = valid_days[valid_days].index  # Get valid (ID, date) pairs
df = df[df.set_index(["ID", "date"]).index.isin(valid_days)]

# Remaining number of days
remaining_days = df.groupby(["ID", df["time"].dt.date]).ngroups
print(f"Remaining number of days after filtering: {remaining_days}")

# Save cleaned data
df.to_csv("data/processed/final_cleaned_cgm.csv", index=False)
print("Data cleaned and saved!")


In [ ]:
# Calculate number of missing values per day
nan_counts = df.groupby(["ID", df["time"].dt.date])["glc"].apply(lambda group: group.isna().sum())

# Summary statistics for NaN counts
print(nan_counts.describe())

# Plot distribution
import matplotlib.pyplot as plt
nan_counts.hist(bins=50)
plt.title("Distribution of Missing Values per Day")
plt.xlabel("Number of Missing Values")
plt.ylabel("Frequency")
plt.show()
